In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime,timedelta

%run /Users/andreas/phys/master/GRB/src/paths/mac.ipynb


All MAC paths are set!


/Users/andreas/fun/useful_py_func/matplotlib_custom_style_and_macros.py:128: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  rcParams['lines.marker'] = None


# Loading trigger times

In [2]:
df_trigtimes = pd.read_csv('trigger_times.txt',sep=' ')
df_trigtimes.drop(columns='Unnamed: 6',axis=0,inplace=True)

In [3]:
df_trigtimes

,year,month,day,hour,minute,second
0,2018,6,1,5,10,4.3
1,2018,6,1,5,12,2.8
2,2018,6,1,5,15,53.1
3,2018,6,1,5,16,35.4
4,2018,6,1,9,54,34.9
...,...,...,...,...,...,...
692439,2022,11,1,23,30,26.7
692440,2022,11,1,23,36,15.8
692441,2022,11,1,23,44,45.8
692442,2022,11,1,23,46,58.5


In [4]:
#Creating datetime object from all rows, then computing timedelta for the relevant entries based on search criteria

In [5]:
datetime_trig = []
year,month,day,hour,minute,second = df_trigtimes['year'].to_numpy(dtype=str),df_trigtimes['month'].to_numpy(dtype=str), df_trigtimes['day'].to_numpy(dtype=str),df_trigtimes['hour'].to_numpy(dtype=str),df_trigtimes['minute'].to_numpy(dtype=str),df_trigtimes['second'].to_numpy(dtype=str)

for i in np.arange(0,len(year)):
    try:
        dt_obj = datetime.strptime(year[i]+'/'+month[i]+'/'+day[i]+' '+hour[i]+':'+minute[i]+':'+second[i],'%Y/%m/%d %H:%M:%S.%f')
        datetime_trig.append(dt_obj)
    except ValueError:
        if second[i] == '60.0':
            second[i] = '59.999'
            dt_obj = datetime.strptime(year[i]+'/'+month[i]+'/'+day[i]+' '+hour[i]+':'+minute[i]+':'+second[i],'%Y/%m/%d %H:%M:%S.%f')
            datetime_trig.append(dt_obj)
print('All arr lengths match: ',len(year)==len(month)==len(day)==len(hour)==len(minute)==len(second)==len(datetime_trig))

All arr lengths match:  True


In [6]:
#not optimal to compute the timedelta for all 600k rows, but there is only a handful of candidates we need to do it for...
#approx 2 min for whole df to compute...
# 982x faster to only compute for the relevant DAY

In [7]:
ASIM_trig_str = '2019/03/05 13:05:19.816'
ASIM_trig_dt = datetime.strptime(ASIM_trig_str,'%Y/%m/%d %H:%M:%S.%f')

In [62]:
def find_bg_match(ASIM_trig_dt=None,datetime_trig=None,time_limit=120.0):
    '''ASIM_trig_dt <datetime> trigger time for ASIM
    datetime_trig <np.arr> contains datetime objects for trigger data
    time_limit <float> limit for the difference between ASIM_trig_dt & datetime_trig. Default 120 s
    
    return <np.array>,<np.array> first array contains time difference and second array is index for trigger in datetime_trig
    '''
    
    trig_day = []
    index_trig_day = []

    for i,dt in enumerate(datetime_trig):
        if dt.year == ASIM_trig_dt.year and dt.month==ASIM_trig_dt.month and dt.day == ASIM_trig_dt.day:
            diff_day = timedelta.total_seconds(datetime_trig[i]-ASIM_trig_dt)
            trig_day.append(diff_day)
            index_trig_day.append(i)
        else:
            continue
    trig_day = np.array(trig_day)
    mask_close = np.where(np.abs(trig_day)<=time_limit)[0]
    print(trig_day[mask_close])
    
    return trig_day[mask_close],index_trig_day[mask_close[0]:mask_close[-1]+1]
    

In [63]:
time_offset, idxs = find_bg_match(ASIM_trig_dt=ASIM_trig_dt,datetime_trig=datetime_trig,time_limit=120.0)

[-91.216   0.284  81.584]


In [64]:
df_trigtimes.iloc[idxs] #~ 1.5 min before trigger

,year,month,day,hour,minute,second
157172,2019,3,5,13,3,48.6
157173,2019,3,5,13,5,20.1
157174,2019,3,5,13,6,41.4


In [34]:
#get data for this trigger using read_matlab_file

#problem accessing the relevant cdf file.. old script works for unzipped files, while now all the cdf
#cdf files have been .gz zipped

#can improve the script be unzipping, but then; where to store the files? -->temp folder on the server
#or stored locally

#ask David tomorrow if he have a good script for reading the files and generating a matlab file
#containing the data